In [1]:
# keras imports for the dataset and building our neural network
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.utils import np_utils

In [2]:
# loading the dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 2s 0us/step


In [3]:
# # building the input vector from the 32x32 pixels
X_train = X_train.reshape(X_train.shape[0], 32, 32, 3)
X_test = X_test.reshape(X_test.shape[0], 32, 32, 3)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [4]:
# normalizing the data to help with the training
X_train = (X_train/255)-0.5
X_test = (X_test/255)-0.5

In [5]:
# one-hot encoding using keras' numpy-related utilities
n_classes = 10
print("no one-hot encoding: ", y_train.shape)
Y_train = np_utils.to_categorical(y_train, n_classes)
Y_test = np_utils.to_categorical(y_test, n_classes)
print("one hot encoding: ", Y_train.shape)

no one-hot encoding:  (50000, 1)
one hot encoding:  (50000, 10)


In [10]:
# building a linear stack of layers with the sequential model
model = Sequential()

# convolutional layer
model.add(Conv2D(50, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', input_shape=(32, 32, 3)))

# convolutional layer
model.add(Conv2D(75, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(125, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# flatten output of conv
model.add(Flatten())

# hidden layer
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(10, activation='softmax'))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 32, 32, 50)        1400      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 75)        33825     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 75)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 16, 16, 75)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 16, 16, 125)       84500     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 8, 8, 125)         0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 8, 8, 125)        

In [11]:
# compiling the sequential model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

# training the model for 10 epochs
model.fit(X_train, Y_train, batch_size=128, epochs=10)

Epoch 1/10
391/391 [==============================] - 7s 19ms/step - loss: 1.4584 - accuracy: 0.4699
Epoch 2/10
391/391 [==============================] - 7s 19ms/step - loss: 0.9998 - accuracy: 0.6461
Epoch 3/10
391/391 [==============================] - 7s 19ms/step - loss: 0.8328 - accuracy: 0.7099
Epoch 4/10
391/391 [==============================] - 7s 19ms/step - loss: 0.7245 - accuracy: 0.7453
Epoch 5/10
391/391 [==============================] - 7s 19ms/step - loss: 0.6448 - accuracy: 0.7755
Epoch 6/10
391/391 [==============================] - 7s 18ms/step - loss: 0.5771 - accuracy: 0.7988
Epoch 7/10
391/391 [==============================] - 7s 19ms/step - loss: 0.5187 - accuracy: 0.8177
Epoch 8/10
391/391 [==============================] - 7s 18ms/step - loss: 0.4651 - accuracy: 0.8359
Epoch 9/10
391/391 [==============================] - 7s 19ms/step - loss: 0.4217 - accuracy: 0.8541
Epoch 10/10
391/391 [==============================] - 7s 19ms/step - loss: 0.3949 - accura

In [8]:
performance = model.evaluate(X_test, Y_test)
print("Accuracy on Test samples: {0}".format(performance[1]))

313/313 [==============================] - 1s 3ms/step - loss: 0.6420 - accuracy: 0.7913
Accuracy on Test samples: 0.7912999987602234
